<a href="https://colab.research.google.com/github/ithelga/bank-churn-predictor/blob/develop/notebooks/Team2_HW2_Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Инжиниринг признаков. Логистическая регрессия

In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from matplotlib.colors import LinearSegmentedColormap
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold, SelectFromModel
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = 'drive/MyDrive/Colab Notebooks/Bank churn predictor/data'
row_df = pd.read_csv(f'{data_path}/row_dataset.csv')
preprocessed_df = pd.read_csv(f'{data_path}/preprocessed_dataset.csv')

# Генерация новых признаков

Будем работать с исходным датасетом для генерации новых признаков, предобработку выполняем аналогично спринту 1

In [ ]:
derived_df = row_df.copy()

# Повторяем преобработку исходного датасета
derived_df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)
derived_df['Geography'].fillna(derived_df['Geography'].mode()[0], inplace=True)
derived_df['Age'].fillna(derived_df['Age'].median(), inplace=True)
derived_df.dropna(inplace=True, ignore_index=True)
derived_df.drop_duplicates(inplace=True, ignore_index=True)

derived_df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42.0,2,0.00,1,1.0,1.0,101348.88,1
1,608,Spain,Female,41.0,1,83807.86,1,0.0,1.0,112542.58,0
2,502,France,Female,42.0,8,159660.80,3,1.0,0.0,113931.57,1
3,699,France,Female,39.0,1,0.00,2,0.0,0.0,93826.63,0
4,645,Spain,Male,44.0,8,113755.78,2,1.0,0.0,149756.71,1


### 1. Создание признаков на основе знаний

Добавим доменные признаки:

* `AgeGroup`: Категоризация возраста на группы (молодые, среднего возраста, пожилые).
* `CreditScoreGroup`: Категоризация кредитного рейтинга (низкий, средний, высокий).
* `TenureGroup`: Категоризация срока удержания клиента в банке на группы (новые, среднесрочные, долгосрочные).
* `ZeroBalance`: Флаг, указывающий, что баланс клиента равен нулю.
* `ActivityScore`: Произведение активности и количества продуктов как мера вовлеченности.
* `TenureToAgeRatio`: Соотношения срока облуживания к возрасту, чтобы оценить долю жизни клиента, проведенную с банком.
* `BalanceToSalaryRatio`: Соотношение баланса к зарплате как индикатор финансового поведения.

In [ ]:
# Категоризация возраста
def age_group(age):
    if age < 30:
        return 'Young'
    elif age < 50:
        return 'Middle'
    else:
        return 'Senior'

derived_df['AgeGroup'] = derived_df['Age'].apply(age_group)

In [ ]:
# Категоризация кредитного рейтинга
def credit_score_group(score):
    if score < 600:
        return 'Low'
    elif score < 700:
        return 'Medium'
    else:
        return 'High'

derived_df['CreditScoreGroup'] = derived_df['CreditScore'].apply(credit_score_group)

In [ ]:
# Категоризация срока удержания клиента
def tenure_group(tenure):
    if tenure <= 2:
        return 'New'
    elif tenure <= 5:
        return 'Medium'
    else:
        return 'LongTerm'

derived_df['TenureGroup'] = derived_df['Tenure'].apply(tenure_group)

In [ ]:
# Флаг нулевого баланса
derived_df['ZeroBalance'] = (derived_df['Balance'] == 0).astype(int)

In [ ]:
# Активность клиента (комбинация активности и количества продуктов)
derived_df['ActivityScore'] = derived_df['IsActiveMember'] * derived_df['NumOfProducts']

In [ ]:
# Соотношение срока удержания клиента к возрасту
derived_df['TenureToAgeRatio'] = derived_df['Tenure'] / (derived_df['Age'] + 1e-5)  # Добавляем малое число для избежания деления на 0

In [ ]:
# Соотношение баланса к зарплате
derived_df['BalanceToSalaryRatio'] = derived_df['Balance'] / (derived_df['EstimatedSalary'] + 1e-5)  # Добавляем малое число для избежания деления на 0

In [ ]:
# Рассмотрим добавленные признаки на 5 произвольных строках
derived_df.sample(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,AgeGroup,CreditScoreGroup,TenureGroup,ZeroBalance,ActivityScore,TenureToAgeRatio,BalanceToSalaryRatio
9662,726,Germany,Male,30.0,7,92847.59,1,1.0,0.0,146154.06,0,Middle,High,LongTerm,0,0.0,0.233333,0.635272
2730,623,Germany,Female,48.0,1,108076.33,1,1.0,0.0,118855.26,1,Middle,Medium,New,0,0.0,0.020833,0.909310
6963,584,Spain,Female,30.0,5,0.00,2,1.0,1.0,185201.58,0,Middle,Low,Medium,1,2.0,0.166667,0.000000
1630,617,France,Male,30.0,3,132005.77,1,1.0,0.0,142940.39,0,Middle,Medium,Medium,0,0.0,0.100000,0.923502
9098,659,France,Male,35.0,6,0.00,2,1.0,1.0,58879.11,0,Middle,Medium,LongTerm,1,2.0,0.171429,0.000000


### 2. Создание признаков на основе взаимодействий

Добавим полиномиальные признаки. Используем `PolynomialFeatures` для создания квадратов и попарных произведений числовых признаков (`Age`^2, `Age` * `Balance` и т.д.):
* `CreditScore` (кредитного рейтинга),
* `Age` (возраста),
* `Balance` (баланса),
* `NumOfProducts` (количества продуктов),
* `EstimatedSalary` (заработной платы).

In [ ]:
# Выделяем числовые признаки для полиномиальных взаимодействий
numeric_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
poly = PolynomialFeatures(degree=(2, 2), include_bias=False, interaction_only=False)
poly_features = poly.fit_transform(derived_df[numeric_features])

In [ ]:
# Названия новых полиномиальных признаков
poly_feature_names = poly.get_feature_names_out(numeric_features)
poly_df = pd.DataFrame(poly_features, columns=poly_feature_names, index=derived_df.index)

In [ ]:
# Объединяем полиномиальные признаки с основным датафреймом
derived_df = pd.concat([derived_df, poly_df], axis=1)

### 3. Обработка добавленных признаков

In [ ]:
# Категориальные признаки
categorical_features = ['Geography', 'AgeGroup', 'CreditScoreGroup', 'TenureGroup']
binary_feature = ['Gender'] # Бинарный признак
# Обновляем список числовых признаков, включая полиномиальные
numeric_features_extended = numeric_features + list(poly_feature_names)

In [ ]:
# Создаем pipeline для обработки
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(sparse_output=False), categorical_features),
        ('num', MinMaxScaler(), numeric_features_extended),
        ('bin', OrdinalEncoder(), binary_feature)
    ])

# Применяем преобразования
derived_df_transformed = preprocessor.fit_transform(derived_df)
# Выделяем целевую переменную
Y = derived_df['Exited']

In [ ]:
# Названия признаков после обработки
bin_feature_names = binary_feature
cat_feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)
all_feature_names = list(cat_feature_names) + numeric_features_extended + bin_feature_names

In [ ]:
# Полученный датасет + целевая переменная
derived_df = pd.DataFrame(derived_df_transformed, columns=all_feature_names)
derived_df = pd.concat([derived_df, Y], axis=1)

# Отбор признаков

Целевая переменная — Exited (бинарная классификация). Так как классы несбалансированы, в качестве метрики будем использовать AUC-ROC (альтерватива - F1-score).

In [ ]:
Y = derived_df['Exited']
X = derived_df.drop(['Exited'], axis=1)
# Разделение на обучающую и тестовую выборки
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

In [ ]:
# Инициализация словарей для хранения отобранных признаков
selected_features = {}

### 1. Статистические методы

In [ ]:
# 1.1 SelectKBest с ANOVA
k_best = SelectKBest(score_func=f_classif, k=10)  # Выбираем топ-10 признаков
k_best.fit(X_train, Y_train)
k_best_features = X_train.columns[k_best.get_support()].tolist()
selected_features['SelectKBest'] = k_best_features

In [ ]:
# 1.2 VarianceThreshold
var_thresh = VarianceThreshold(threshold=0.1)  # Удаляем признаки с дисперсией < 0.1
var_thresh.fit(X_train)
var_thresh_features = X_train.columns[var_thresh.get_support()].tolist()
selected_features['VarianceThreshold'] = var_thresh_features

### 2. Отбор на основе моделей

In [ ]:
# 2.1 SelectFromModel с логистической регрессией (L1-регуляризация)
lr_model = LogisticRegression(penalty='l1', solver='liblinear', C=1.0, random_state=42)
lr_selector = SelectFromModel(lr_model, max_features=10)
lr_selector.fit(X_train, Y_train)
lr_features = X_train.columns[lr_selector.get_support()].tolist()
selected_features['LogisticRegression'] = lr_features

In [ ]:
# 2.2 SelectFromModel с Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_selector = SelectFromModel(rf_model, max_features=10)
rf_selector.fit(X_train, Y_train)  # Random Forest не требует масштабирования
rf_features = X_train.columns[rf_selector.get_support()].tolist()
selected_features['RandomForest'] = rf_features

### 3. Sequential Feature Selection (SFS)

In [ ]:
sfs_model = LogisticRegression(random_state=42)
sfs = SFS(sfs_model,
          k_features=10,  # Выбираем 10 признаков
          forward=True,   # Прямой отбор
          floating=False,
          scoring='roc_auc',
          cv=5,
          n_jobs=-1)
sfs.fit(X_train, Y_train)
sfs_features = X_train.columns[list(sfs.k_feature_idx_)].tolist()
selected_features['SFS'] = sfs_features

### 4. Оценка результатов

In [ ]:
# Вывод результатов
print("Отобранные признаки для каждого метода:")
for method, features in selected_features.items():
    print(f"\n{method}: {len(features)} признаков")
    print(features)

Отобранные признаки для каждого метода:

SelectKBest: 10 признаков
['Geography_Germany', 'AgeGroup_Senior', 'AgeGroup_Young', 'Age', 'Balance', 'CreditScore Age', 'Age^2', 'Age Balance', 'Age EstimatedSalary', 'Balance NumOfProducts']

VarianceThreshold: 13 признаков
['Geography_France', 'Geography_Germany', 'Geography_Spain', 'AgeGroup_Middle', 'AgeGroup_Senior', 'AgeGroup_Young', 'CreditScoreGroup_High', 'CreditScoreGroup_Low', 'CreditScoreGroup_Medium', 'TenureGroup_LongTerm', 'TenureGroup_Medium', 'TenureGroup_New', 'Gender']

LogisticRegression: 10 признаков
['Geography_France', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'CreditScore EstimatedSalary', 'Age^2', 'Balance^2', 'Balance NumOfProducts', 'NumOfProducts^2']

RandomForest: 10 признаков
['Age', 'NumOfProducts', 'CreditScore Age', 'CreditScore NumOfProducts', 'Age^2', 'Age Balance', 'Age NumOfProducts', 'Balance NumOfProducts', 'NumOfProducts^2', 'NumOfProducts EstimatedSalary']

SFS: 10 признаков
['Geography_Germany', 'Ag

In [ ]:
# Оценка логистической регрессии на отобранных признаках
results = {}
for method, features in selected_features.items():
    # Выбираем подмножество столбцов
    X_train_subset = X_train[features]
    X_test_subset = X_test[features]

    model = LogisticRegression(random_state=42)
    model.fit(X_train_subset, Y_train)
    Y_pred = model.predict(X_test_subset)
    roc_auc = roc_auc_score(Y_test, Y_pred)
    results[method] = roc_auc

In [ ]:
# Вывод результатов оценки
print("ROC-AUC логистической регрессии на тестовом наборе:")
for method, roc_auc in results.items():
    print(f"{method}: {roc_auc:.4f}")

ROC-AUC логистической регрессии на тестовом наборе:
SelectKBest: 0.5424
VarianceThreshold: 0.5372
LogisticRegression: 0.5369
RandomForest: 0.5314
SFS: 0.5648


По результатам оценки ROC_AUC лучший результат показал набор признаков, полученный при помощи SFS.

In [ ]:
# Полученный датафрейм
extract_df = pd.concat([derived_df[selected_features['SFS']], Y], axis=1)
extract_df.head()

,Geography_Germany,Age,Balance,NumOfProducts,CreditScore NumOfProducts,Age^2,Age Balance,Balance NumOfProducts,NumOfProducts^2,Gender,Exited
0,0.0,0.324324,0.000000,0.000000,0.088197,0.176904,0.000000,0.000000,0.000000,0.0,1
1,0.0,0.310811,0.334031,0.000000,0.084590,0.166708,0.249049,0.107315,0.000000,0.0,0
2,0.0,0.324324,0.636357,0.666667,0.379016,0.176904,0.486031,0.613331,0.533333,0.0,1
3,0.0,0.283784,0.000000,0.333333,0.343607,0.147052,0.000000,0.000000,0.200000,0.0,0
4,0.0,0.351351,0.453394,0.333333,0.308197,0.198034,0.362780,0.291325,0.200000,1.0,1


# Обучение модели Логистической регрессии

Обучение на исходных данных

In [ ]:
# Размер выборки
row_df.shape

(10002, 14)

In [ ]:
# Проверка дисбаланса классов
disbalance_ratio = row_df['Exited'].value_counts(normalize=True)
print(f"Соотношение классов:\n{disbalance_ratio}")

Соотношение классов:
Exited
0    0.796241
1    0.203759
Name: proportion, dtype: float64


In [ ]:
def evaluate_model(df, dataset_name, is_raw_data=False):
    """
    Обучение и оценка модели логистической регрессии.

    Параметры:
    - df: DataFrame с данными
    - dataset_name: название датасета
    - is_raw_data: если True, удаляет категориальные признаки (только для row_df)

    Возвращает:
    - Словарь с метриками и названием датасета
    """
    # Копирование данных для сохранения исходных
    data = df.copy()

    # Удаление технических колонок
    non_features = ['RowNumber', 'CustomerId', 'Surname']
    if 'Exited' in data.columns:
        target = data['Exited']
        features = data.drop(columns=['Exited'] + [col for col in non_features if col in data.columns])
    else:
        raise ValueError("Столбец 'Exited' не найден в данных.")

    # Минимальная обработка для исходных данных
    if is_raw_data:
        # Удаление категориальных признаков
        cat_cols = features.select_dtypes(include=['object', 'category']).columns
        features = features.drop(columns=cat_cols)
        # Заполнение пропущенных значений медианой
        features = features.fillna(features.median())

    # Разделение данных
    X_train, X_test, y_train, y_test = train_test_split(
        features, target,
        test_size=0.25,
        random_state=42,
        stratify=target # сохраняет соотношение классов
    )

    # Обучение модели
    model = LogisticRegression(
        class_weight='balanced', # учёт дисбаланса
        max_iter=500,
        solver='lbfgs',
        random_state=42
    )
    model.fit(X_train, y_train)

    # Предсказания
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    # Метрики
    metrics = {
        'Dataset': dataset_name,
        'F1': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_proba),
        'Recall': recall_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred)
    }

    return metrics

In [ ]:
# Список датасетов и их характеристик
datasets = [
    {'data': row_df, 'name': 'Первоначальные данные', 'is_raw': True},
    {'data': preprocessed_df, 'name': 'Предобработанные данные', 'is_raw': False},
    {'data': derived_df, 'name': 'Сгенерированные признаки', 'is_raw': False},
    {'data': extract_df, 'name': 'Отобранные признаки', 'is_raw': False}
]

# Сбор результатов
results = []
for dataset in datasets:
    metrics = evaluate_model(
        dataset['data'],
        dataset['name'],
        is_raw_data=dataset['is_raw']
    )
    results.append(metrics)

# Итоговая таблица
results_df = pd.DataFrame(results).set_index('Dataset')

In [ ]:
# Вывод результата
colors = ["#FFAFCC", "#FFC8DD", "#CDB4DB", "#BDE0FE", "#A2D2FF"]
cmap = LinearSegmentedColormap.from_list("custom", colors)

def colorize(val):
    if val > 0.8:
        return f'background-color: {colors[4]}; font-weight: bold'
    elif val > 0.7:
        return f'background-color: {colors[3]}; font-weight: bold'
    elif val > 0.6:
        return f'background-color: {colors[2]}; font-weight: bold'
    elif val > 0.5:
        return f'background-color: {colors[1]}; font-weight: bold'
    else:
        return f'background-color: {colors[0]}; font-weight: bold'

styled_df = results_df.style\
    .format("{:.3f}")\
    .applymap(colorize)\
    .set_properties(**{
        'text-align': 'center',
        'font-size': '12pt',
        'font-weight': 'normal',
        'border': '1px solid white'
    })\
    .set_table_styles([{
        'selector': 'th',
        'props': [('background-color', 'white'),
                 ('color', 'black'),
                 ('text-align', 'left'),
                 ('font-weight', 'normal'),
                 ('font-size', '12pt')]
    }])\
    .background_gradient(cmap=cmap)

styled_df

,F1,ROC-AUC,Recall,Precision
Dataset,,,,
Первоначальные данные,0.458,0.747,0.690,0.342
Предобработанные данные,0.507,0.783,0.705,0.396
Сгенерированные признаки,0.539,0.815,0.715,0.433
Отобранные признаки,0.547,0.818,0.741,0.434
